In [0]:
#colab 을 이용한 실행시
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torchtext.data as data
import torchtext.datasets as datasets
import pickle
print (torch.__version__)

1.3.1


In [0]:
class RNN_Text(nn.Module):
    
    def __init__(self, embed_num, class_num):
        super(RNN_Text, self).__init__()
        V = embed_num
        C = class_num
        H = 256
        
        self.embed = nn.Embedding(V, 100)
        self.rnn = nn.LSTM(100,H, bidirectional = True)
        self.out = nn.Linear(H*2, C)
        
    def forward(self, x):
        x = self.embed(x)  # (N, W, D)
        #x = x.unsqueeze(1)  # (N, Ci, W, D)
        x,(_,__) = self.rnn( x, ( self.h, self.c ) )
        
        logit = self.out(x[-1])
        return logit
    def inithidden(self,b):
        self.h = Variable(torch.randn(2, b, 256))
        self.c = Variable(torch.randn(2, b, 256))
        

In [0]:
class mydataset(data.Dataset):
    @staticmethod
    def sort_key(ex):
        return len(ex.text)
    def __init__(self, text_field, label_field, path=None, examples=None, **kwargs):
        fields = [('text', text_field), ('label', label_field)]
        if examples is None:
            path = self.dirname if path is None else path
            examples = []
            for i,line in enumerate(open(path,'r',encoding='utf-8')):
                if i==0:
                    continue
                line = line.strip().split('\t')
                
                txt= [ d.split('/')[0] for d in line[1].split(' ') ]
                examples += [ data.Example.fromlist( [txt, line[2]],fields ) ]
        super(mydataset, self).__init__(examples, fields, **kwargs)

In [0]:
text_field = data.Field()
label_field = data.Field(sequential=False, batch_first = True, unk_token = None)

train_data = mydataset(text_field,label_field,path='/content/gdrive/My Drive/Colab Notebooks/nsm/twit_ratings_train.txt')

test_data = mydataset(text_field,label_field,path='/content/gdrive/My Drive/Colab Notebooks/nsm/twit_ratings_test.txt')

text_field.build_vocab(train_data)
label_field.build_vocab(train_data)

train_iter, test_iter = data.Iterator.splits(
                            (train_data, test_data), 
                            batch_sizes=(100, 1), repeat=False)
len(text_field.vocab)

61490

In [0]:
rnn = torch.load('/content/gdrive/My Drive/Colab Notebooks/pre_models/rnn_model_large.pt', map_location=lambda storage, loc: storage)

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:493: SourceChangeWarning: source code of class 'torch.nn.modules.sparse.Embedding' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:493: SourceChangeWarning: source code of class 'torch.nn.modules.rnn.LSTM' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:493: SourceChangeWarning: source code of class 'torch.nn.modules.linear.Linear' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn

In [0]:
%%time

correct = 0
incorrect = 0
for batch in test_iter:
    txt = batch.text
    label = batch.label
    
    rnn.inithidden(txt.size(1))
    
    pred = rnn(txt)
    _,ans = torch.max(pred,dim=1)
    
    if ans.data[0] == label.data[0]:
        correct += 1    
    else:
        incorrect += 1
    
print ('correct : ', correct)
print ('incorrect : ', incorrect)
print ()


correct :  42276
incorrect :  7724

CPU times: user 3min 14s, sys: 492 ms, total: 3min 14s
Wall time: 3min 15s
